In [1]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
# %tensorflow_version 2.x
# import tensorflow as tf
# tf.test.gpu_device_name()
# from scipy import integrate
# import os
# import numpy as np
# from tensorflow import keras
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K


from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr.clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot



import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from attention import Attention

C:\Users\moshi\AppData\Local\Temp/ipykernel_2736/628588012.py:43: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [6]:
data_train = pd.read_excel('train_nor_811.xlsx') 
data_test = pd.read_excel('test_nor_811.xlsx')
data_val= pd.read_excel('valid_nor_811.xlsx')




data = data_train.append(data_test, ignore_index=True)
temp= data.append(data_val, ignore_index=True)


print(temp.Emotion.value_counts())

temp.head(10)


Enjoyment    1965
Disgust      1338
Other        1291
Sadness      1149
Anger         480
Fear          395
Surprise      309
Name: Emotion, dtype: int64


,Unnamed: 0,Emotion,Sentence
0,188,Other,cho mình xin bài nhạc tên là gì với ạ
1,166,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,1345,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,316,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,1225,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...
5,1220,Anger,thằng kia sao mày bắt vợ với bồ tao dọn thế ki...
6,44,Other,một lí do trog muôn vàn lí do
7,1951,Surprise,thật hay đùa ác vậy . không thể tin được
8,1249,Anger,"ko phải con mình , mà xem còn thấy đau như vậy..."
9,1063,Sadness,per nghe đi rồi khóc 1 trận cho thoải mái . đừ...


In [8]:
train, test = train_test_split(temp, test_size=0.2, stratify = temp['Emotion'], random_state = 42)
num_classes = 7
embed_num_dims = 300
max_seq_len = 50

x_train = train['Sentence']
x_test = test['Sentence']

y_train = train['Emotion']
y_test = test['Emotion']

texts_train = x_train
texts_test = x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['Sentence'])

sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index

vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len, padding='pre' )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len,  padding='pre')

print(X_train_pad)


encoding = {
    'Enjoyment': 0,
    'Disgust': 1,
    'Other': 2,
    'Sadness': 3,
    'Anger': 4,
    'Fear':5,
    'Surprise':6
}

y_train = [encoding[x] for x in train['Emotion']]
y_test = [encoding[x] for x in test['Emotion']]



y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('cc.vi.300.vec',encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)
print(embedd_matrix.shape)




Number of unique words: 4430
[[   0    0    0 ...  119   38   22]
 [   0    0    0 ...  405 2759  308]
 [   0    0    0 ...  203   20   17]
 ...
 [   0    0    0 ...  350 2023 4430]
 [ 151  100   54 ...   26   18   34]
 [   0    0    0 ...  100  836   67]]
(4431, 300)


# Random Search


In [9]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =32, max_value = 128, step = 16)
  fake_val = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =64, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =64, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)



  seq_input = keras.layers.Input(shape=(max_seq_len,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = max_seq_len,
                          weights = [embedd_matrix])(seq_input)

  cnn = keras.layers.Conv1D(cnn_1_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)

  attention_vec = keras.layers.TimeDistributed(keras.layers.Dense(unit_attention))(cnn)
  attention_vec = keras.layers.Reshape((48,unit_attention))(attention_vec)
  attention_vec = keras.layers.Activation('relu', name = 'cnn_attention_vec')(attention_vec)
  attention_output = keras.layers.Dot(axes = 1)([cnn, attention_vec])


  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_unit, recurrent_regularizer=regularizers.l2(1e-4),
                                                      return_sequences=True,kernel_regularizer=regularizers.l2(1e-4),
                                                      bias_regularizer=regularizers.l2(1e-2),
                                                      activity_regularizer=regularizers.l2(1e-4),input_shape =(48,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  cnn_2 = keras.layers.Conv1D(cnn_2_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(num_classes, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  return model


stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


clr_step_size = int((len(X_train_pad)/64))
base_lr = 1e-3
max_lr = 6e-3
mode = 'exp_range'


clr = CyclicLR(base_lr = base_lr, max_lr = max_lr, step_size = clr_step_size, mode = mode)



tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 50,
    executions_per_trial = 1,
    directory = LOG_DIR
    )
  
tuner.search(x=X_train_pad,y = y_train,epochs = 20, batch_size = 128,callbacks = [stop,clr], 
             validation_data = (X_test_pad,y_test))


tuner.results_summary()

Trial 50 Complete [00h 05m 44s]
val_accuracy: 0.51875901222229

Best val_accuracy So Far: 0.5418470501899719
Total elapsed time: 05h 18m 00s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in 1646769255\untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
attention_unit: 80
cnn_1_unit: 80
cnn_1_dropout: 0.2
lstm_unit: 192
lstm_dropout: 0.1
cnn_2_unit: 64
cnn_2_dropout: 0.2
Score: 0.5418470501899719
Trial summary
Hyperparameters:
attention_unit: 48
cnn_1_unit: 32
cnn_1_dropout: 0.2
lstm_unit: 96
lstm_dropout: 0.30000000000000004
cnn_2_unit: 224
cnn_2_dropout: 0.1
Score: 0.5295815467834473
Trial summary
Hyperparameters:
attention_unit: 96
cnn_1_unit: 64
cnn_1_dropout: 0.30000000000000004
lstm_unit: 160
lstm_dropout: 0.4
cnn_2_unit: 96
cnn_2_dropout: 0.30000000000000004
Score: 0.5281385183334351
Trial summary
Hyperparameters:
attention_unit: 80
cnn_1_unit: 64
cnn_1_dropout: 0.2
lstm_unit: 96
lstm_dropout: 0.2
